**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


█████████████████████████████████████████▌                                      

**Lendo tabela da bronze**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.1_bronze.tests").load()

df.printSchema()

root
 |-- tat_hours: long (nullable = true)
 |-- sample: string (nullable = true)
 |-- method: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- active: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- ref_range: struct (nullable = true)
 |    |-- high: double (nullable = true)
 |    |-- low: double (nullable = true)
 |-- test_code: string (nullable = true)



**Selecionando colunas necessarias do dataframe.**

In [ ]:
df_tratado_v1 = (df.select(
    "tat_hours",
    "sample",
    "method",
    "unit",
    "active",
    "name",
    "price",
    "ref_range.*",
    "test_code"
    )
)

**Verificando se há nulos nas colunas.**

In [ ]:
df_tratado_v1.select([F.sum(F.col(c).isNull().cast("integer")).alias(c) for c in df_tratado_v1.columns]).show()

+---------+------+------+----+------+----+-----+----+---+---------+
|tat_hours|sample|method|unit|active|name|price|high|low|test_code|
+---------+------+------+----+------+----+-----+----+---+---------+
|        0|     0|     0|   0|     0|   0|    0|   0|  0|        0|
+---------+------+------+----+------+----+-----+----+---+---------+



**Salvando dataframe na silver.**

In [ ]:
(df_tratado_v1.write
        .format("bigquery")
        .option("table", "clinic-de.2_silver.tests")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("----------------------Tabela salva com sucesso----------------------")

----------------------Tabela salva com sucesso----------------------


In [ ]:
spark.stop()